In [2]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
data = pd.concat(( train, test ))
train = []
test = []

In [4]:
print("msno")
msno_values = data.msno.unique()
print("end msno")
del(data)

msno
end msno


In [5]:
counts = pd.read_csv("cleaned_input/counts.csv")

In [8]:
#get number of unique visits
user_features = pd.DataFrame() 

user_features['msno'] = np.nan #counts.index
#user_features["frequency"] = np.nan #counts.values
user_features["gap_days"] = np.nan
user_features["average_gap_days"] = np.nan
user_features["longest_gap"] = np.nan
user_features["gaps_longer_29"] = np.nan
user_features.head()

,msno,gap_days,average_gap_days,longest_gap,gaps_longer_29


In [10]:
#read in user_logs
import time
chunksize = 25000
i = 0 
for chunk in pd.read_csv('raw_data/user_logs.csv', engine='c', chunksize=chunksize, usecols=[0, 1, 7, 8], dtype={"msno":str, "date": int, "num_unq":int, "total_secs":float}):
    if i > 10000:
        start_time = time.time()
        chunk = chunk[chunk['msno'].isin(msno_values)]
        rows = chunk.msno.unique()
        chunk['date'] = pd.to_datetime(chunk['date'], format='%Y%m%d')
        #print("processing ",i,"took %s seconds" % (time.time() - start_time))
        #print(len(rows))
        for row in rows:
            row_time = time.time()
            df = chunk[chunk["msno"] == row]
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            differences = df.date.diff()
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            gap_days = differences.sum()
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            average_gap_days = differences.mean()
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            if len(differences.nlargest(1)) > 0:
                longest_gap = differences.nlargest(1).iloc[0].total_seconds()
            else:
                longest_gap = -1
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            gaps_longer_29 = 0
            for val in differences:
                if val > pd.Timedelta('29 days'):
                    gaps_longer_29 += 1
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
            user_features.loc[len(user_features)] = [row, gap_days, average_gap_days.total_seconds(), longest_gap, gaps_longer_29]
            #print("row ",i,"took %s seconds" % (time.time() - row_time))
        #del(chunk)
        print("Round ",i,"took %s seconds" % (time.time() - start_time))
        if i % 100 == 0:
            user_features.to_csv(str(i) + "_user_logs_features_4.csv")
        i += 1 
user_features.head()

,msno,gap_days,average_gap_days,longest_gap,gaps_longer_29
